In [1]:
# Дисклеймер:
# что делает код - написано комментариями в ячейках кода.
# Код написан не для продакшена, а на "скорую руку" для анализа

# 1. Импортирование библиотек

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# 2. Загрузка данных

In [3]:
prolongations = pd.read_csv('data/prolongations.csv')
financial_data = pd.read_csv('data/financial_data.csv')

**Описание колонок**:

**prolongations.csv**:
- id – id проекта;
- month – последний месяц реализации проекта;
- AM – ФИО ответственного аккаунт-менеджера (данные первичны по отношению к financial_data).

**financial_data.csv**:
- id – id проекта;
- Причина дубля – причина, почему строки с одним и тем же id встречаются несколько раз;
- Колонки с названием месяца – сумма отгрузки проекта в данный месяц;
- Account – ФИО ответственного аккаунт-менеджера.

В колонках с названием месяца таблицы *financial_data* могут также встречаться такие значения: ‘в ноль’ –  отгрузка проекта в данном месяце равна 0, значит для коэффициента пролонгации нужно взять отгрузку предыдущего месяца (только если все части оплаты равны 0); ‘стоп’ – проект закончился до истечения срока договора, если у проекта есть “стоп” в последний месяц реализации или ранее, то такой проект исключаем из пролонгаций; ‘end’ – аналогично ‘стоп’

# 3. Предобработка данных

### 3.1. Анализ таблицы `prolongations`

In [4]:
# посмотрим на данные 
prolongations.head()

,id,month,AM
0,42,ноябрь 2022,Васильев Артем Александрович
1,453,ноябрь 2022,Васильев Артем Александрович
2,548,ноябрь 2022,Михайлов Андрей Сергеевич
3,87,ноябрь 2022,Соколова Анастасия Викторовна
4,429,ноябрь 2022,Соколова Анастасия Викторовна


In [5]:
# посмотрим на сводку датафрейма
prolongations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      477 non-null    int64 
 1   month   477 non-null    object
 2   AM      477 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.3+ KB


In [6]:
# посмотрим на количество уникальных элементов по столбцам
prolongations.nunique()

id       313
month     14
AM        10
dtype: int64

In [7]:
# посмотрим на дубликаты
prolongations.duplicated().sum()

np.int64(3)

In [8]:
# посмотрим на уникальные данные по всем колонкам, чтобы выявить необычные значения
for i in prolongations.columns:
    print(prolongations[i].unique())

[  42  453  548   87  429  600  657  594  665  586  637  419  578  579
  671  633  645  369  547  592  666  620  301  704   15   16  379   31
  572  573  574  575  576  448  492  597  708  719   39  328  716  613
  615  617  643   45  318   46   49  603  441  461  717  112  630  154
  418  456  457  454  455  693  654  475  193  390  477  478  535  709
  581  623  196  199  705  629  720  228  242  471  596  714  266  498
  703  524  274  281  583  697  626  684  493  674  728  624  107  721
  509  649  687  519  230  732  529  348  381  556  640  101  690  653
  744  174  667  180  190  691  273  785  530  549  686  726   56  783
  692  647  683  506  540  646  768  713  244  562  682  580  789  797
  798  799  800   48  495  787  675  663  743  781  782  740  788  712
  825  361  804  838  771  765  294  814  796  853  563  829  837  775
  813  842  843  844  845  855  585  569  711  832   55  792  847  337
  858  610  857  869  807  884  806  871  553  848  346  830  727  211
  808 

**Вывод по `prolongations`:**

В таблице `prolongations` нет пропущенных значений. Колонка `month` имеет тип `object`, в будущем необходимо будет поменять эту колонку на тип дата. Имеюся дубликаты, которые скоро удалим. Также среди менеджеров есть непонятное имя 'без А/М', которое стот удалить.

### 3.2. Предобработка таблицы `prolongations`

In [9]:
# удалим из таблицы неизвестных менеджеров
prolongations = prolongations[prolongations['AM']!='без А/М']
# также удалим дубликаты
prolongations = prolongations.drop_duplicates()

### 3.3 Анализ таблицы `financial_data`

In [10]:
# посмотрим на данные
financial_data.head()

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
0,42,NaN,"36 220,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
3,594,NaN,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
4,665,NaN,"10 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович


In [11]:
# посмотрим на общую сводку
financial_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             451 non-null    int64 
 1   Причина дубля  301 non-null    object
 2   Ноябрь 2022    156 non-null    object
 3   Декабрь 2022   159 non-null    object
 4   Январь 2023    139 non-null    object
 5   Февраль 2023   145 non-null    object
 6   Март 2023      168 non-null    object
 7   Апрель 2023    174 non-null    object
 8   Май 2023       190 non-null    object
 9   Июнь 2023      190 non-null    object
 10  Июль 2023      195 non-null    object
 11  Август 2023    199 non-null    object
 12  Сентябрь 2023  186 non-null    object
 13  Октябрь 2023   182 non-null    object
 14  Ноябрь 2023    171 non-null    object
 15  Декабрь 2023   146 non-null    object
 16  Январь 2024    95 non-null     object
 17  Февраль 2024   101 non-null    object
 18  Account        451 non-null   

In [12]:
# посмотрим на количество пропущенных значений по всем колонкам
financial_data.isna().sum()

id                 0
Причина дубля    150
Ноябрь 2022      295
Декабрь 2022     292
Январь 2023      312
Февраль 2023     306
Март 2023        283
Апрель 2023      277
Май 2023         261
Июнь 2023        261
Июль 2023        256
Август 2023      252
Сентябрь 2023    265
Октябрь 2023     269
Ноябрь 2023      280
Декабрь 2023     305
Январь 2024      356
Февраль 2024     350
Account            0
dtype: int64

In [13]:
# посмотрим есть ли дубликаты
financial_data.duplicated().sum()

np.int64(0)

In [14]:
# посмотрим на уникальные данные по всем колонкам, чтобы выявить необычные значения
for i in financial_data.columns:
    print(financial_data[i].unique())

[  42  657  594  665  637  419  578  579  592  666  620  704   16  379
   31  572  573  574  575  576  328  716  613  615  617  643  318  603
  461  717  418  456  457  454  455  693  654  475  193  390  477  478
  535  709  581  623  705  720  242  596  266  498  524  626  684  548
  493  674  624  721  509  687  230  703  529  453  429  381  556  640
  101  653  174  714  649  671  732  530  549  686   56  692  647  683
  506  540  646  697  799  800  675  663  788  728  667  825  838  765
  294  814  796  853  563  837  775  813  842  843  844  845  301  855
  583  585   15  711  832   55  441  847   87  600  787  858  857  869
  807  884  691  806  740  871  495  553  346  586  830  727  211  823
  786  891  904  901  907  897  580  719  848  337  180  854  803  648
  268  274  781  916  923  633  713  829  846  348  492  819  726  792
  898   86  676  876  877  878  668  821  921  920  361  912  872  969
  789  771  859  960  865  874  946  831  782  933   83  460  963  841
  822 

**Вывод по `financial_data`:**

В таблице `financial_data` во всех колонках, за исключением колонок `id` и `Account`, есть пропущенные значения. Среди менеджеров есть непонятное имя 'без А/М', которое стот удалить. Тип всех транзакций представлен не в числовом типе. В колонках транзакций есть три интересных значений - *end*, *стоп* и *в ноль*. 

### 3.4 Предобработка таблицы `financial_data`

In [15]:
# добавим временную колонку в которой будет два значения, если в строке есть 
# один из значений end или стоп, то записываем stop иначе no stop
financial_data['stop_or_no'] = financial_data.apply(lambda x: 'stop' if x.isin(['стоп', 'end']).any() else 'no stop', axis=1)

In [16]:
# посмотрим что получилось
financial_data.head()

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account,stop_or_no
0,42,NaN,"36 220,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович,no stop
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович,stop
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович,stop
3,594,NaN,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович,stop
4,665,NaN,"10 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович,no stop


In [17]:
# удалим строки где среди транзакций есть значения end и стоп зашифрованные под именем stop
financial_data = financial_data[financial_data['stop_or_no']!='stop']

In [18]:
# удалим временную колонку
financial_data = financial_data.drop('stop_or_no', axis=1)

In [19]:
# создадим вертикальную таблицу из таблицы financial_data и сохраним её в ту же переменную
id_vars=['id', 'Причина дубля', 'Account']
value_vars=[column for column in financial_data.columns if column not in id_vars]
financial_data = pd.melt(financial_data, id_vars=id_vars, value_vars=value_vars, var_name='payment_date', value_name='payment')

In [20]:
# посмотрим на результат
financial_data.head()

,id,Причина дубля,Account,payment_date,payment
0,42,NaN,Васильев Артем Александрович,Ноябрь 2022,"36 220,00"
1,665,NaN,Васильев Артем Александрович,Ноябрь 2022,"10 000,00"
2,637,NaN,Соколова Анастасия Викторовна,Ноябрь 2022,"38 045,00"
3,419,NaN,Михайлов Андрей Сергеевич,Ноябрь 2022,NaN
4,578,NaN,Попова Екатерина Николаевна,Ноябрь 2022,"82 800,00"


In [21]:
# посмотрим на результат
financial_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6064 entries, 0 to 6063
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             6064 non-null   int64 
 1   Причина дубля  4032 non-null   object
 2   Account        6064 non-null   object
 3   payment_date   6064 non-null   object
 4   payment        2236 non-null   object
dtypes: int64(1), object(4)
memory usage: 237.0+ KB


In [22]:
# посмотрим на количество пропущенных значений
financial_data.isna().sum()

id                  0
Причина дубля    2032
Account             0
payment_date        0
payment          3828
dtype: int64

Есть пропущенные значения в колонках `Причина дубля` и `payment`. Нас больше интересует колонка `payment`, поэтому займемся им.

In [23]:
# найдем индекс строк с пропущенными значениями колонки payment
financial_data_index = financial_data[financial_data['payment'].isna()].index
# удалим эти найденные строки
financial_data = financial_data.drop(financial_data_index)
# также удалим строки с неизвестным менеджером
financial_data = financial_data[financial_data['Account']!='без А/М']

### 3.5 Объединение двух таблиц

In [24]:
financial_prolongations = pd.merge(prolongations, financial_data, on='id', how='inner')
financial_prolongations.head()

,id,month,AM,Причина дубля,Account,payment_date,payment
0,42,ноябрь 2022,Васильев Артем Александрович,NaN,Васильев Артем Александрович,Ноябрь 2022,"36 220,00"
1,453,ноябрь 2022,Васильев Артем Александрович,NaN,Васильев Артем Александрович,Ноябрь 2022,в ноль
2,453,ноябрь 2022,Васильев Артем Александрович,NaN,Васильев Артем Александрович,Декабрь 2022,"39 245,00"
3,453,ноябрь 2022,Васильев Артем Александрович,NaN,Васильев Артем Александрович,Январь 2023,"44 320,00"
4,453,ноябрь 2022,Васильев Артем Александрович,NaN,Васильев Артем Александрович,Февраль 2023,"177 635,00"


### 3.6. Предобработка объединенной таблицы

In [25]:
# избавимся от двух ненужных для анализ колонок
financial_prolongations = financial_prolongations.drop(['Причина дубля', 'Account'], axis=1)

# изменим тип даты завершения проекта:
# создадим диапазон дат завершения проекта
month_new = pd.date_range(start='2022-11-01', end='2023-12-01', freq='MS').to_list()
# узнаем уникальные месяцы завершения проекта
month_old = list(financial_prolongations.month.unique())
# и наконец заменим тип данных
financial_prolongations['month'] = financial_prolongations['month'].apply(lambda x: month_new[month_old.index(x)])

# сделаем то же самое для даты платежей
payment_new = pd.date_range(start='2022-11-01', end='2024-02-01', freq='MS').to_list()
payment_old = list(financial_prolongations.payment_date.unique())
financial_prolongations['payment_date'] = financial_prolongations['payment_date'].apply(lambda x: payment_new[payment_old.index(x)])

# заменим значения "в ноль" на 0. Это нам пригодится в будущем
financial_prolongations['payment'] = financial_prolongations['payment'].replace('в ноль', '0')

# заменим пробелы и запятые для дальнейшей обработки
financial_prolongations['payment'] = financial_prolongations['payment'].str.replace('\xa0', '').str.replace(',', '.')
# теперь заменим тип платежей на float
financial_prolongations.payment = financial_prolongations.payment.astype('float')

In [26]:
# посмотрим на объединенную таблицу после обработки
financial_prolongations.head()

,id,month,AM,payment_date,payment
0,42,2022-11-01,Васильев Артем Александрович,2022-11-01,36220.0
1,453,2022-11-01,Васильев Артем Александрович,2022-11-01,0.0
2,453,2022-11-01,Васильев Артем Александрович,2022-12-01,39245.0
3,453,2022-11-01,Васильев Артем Александрович,2023-01-01,44320.0
4,453,2022-11-01,Васильев Артем Александрович,2023-02-01,177635.0


In [27]:
# преобразуем таблицу в новую, с уникальными платежами, сгруппированную по четырём колонкам
financial_prolongations = financial_prolongations.groupby(['id',	'month',	'AM', 'payment_date'], as_index=False)['payment'].unique()

In [28]:
# посмотрим на конец получившейся таблицы
financial_prolongations.tail(10)

,id,month,AM,payment_date,payment
3072,998,2023-11-01,Попова Екатерина Николаевна,2024-01-01,"[41050.0, 0.0]"
3073,998,2023-11-01,Попова Екатерина Николаевна,2024-02-01,"[46120.0, 425.0]"
3074,1001,2023-11-01,Кузнецов Михаил Иванович,2023-11-01,[290000.0]
3075,1006,2023-12-01,Смирнова Ольга Владимировна,2023-10-01,[145195.0]
3076,1006,2023-12-01,Смирнова Ольга Владимировна,2023-11-01,[140495.0]
3077,1006,2023-12-01,Смирнова Ольга Владимировна,2023-12-01,[140495.0]
3078,1012,2023-11-01,Петрова Анна Дмитриевна,2023-11-01,[98492.0]
3079,1012,2023-11-01,Петрова Анна Дмитриевна,2023-12-01,[109442.52]
3080,1015,2023-11-01,Смирнова Ольга Владимировна,2023-11-01,"[103700.0, 60000.0]"
3081,1015,2023-11-01,Смирнова Ольга Владимировна,2023-12-01,[15000.0]


Этот шаг был сделан для следующего. Если в списке есть два или более значений и среди них есть ноль, то мы сложим значения списка, так как это отноится к одному заказу, месяцу завершения проекта и месяцу платежа. А где в списке будет только одно значение и оно ноль, то так и оставим это значение.

In [29]:
# реализуем вышесказанное
financial_prolongations['payment'] = financial_prolongations['payment'].apply(lambda x: sum(x))

In [30]:
# посмотрим что получилось
financial_prolongations.head()

,id,month,AM,payment_date,payment
0,15,2022-12-01,Иванова Мария Сергеевна,2022-11-01,439280.00
1,15,2022-12-01,Иванова Мария Сергеевна,2022-12-01,439280.00
2,15,2022-12-01,Иванова Мария Сергеевна,2023-01-01,102433.75
3,15,2022-12-01,Иванова Мария Сергеевна,2023-02-01,102433.75
4,15,2022-12-01,Иванова Мария Сергеевна,2023-03-01,102433.75


Теперь заменим оставшиеся нули(которые `в ноль`) значением продажи за предыдущий месяц.

In [31]:
# для начала отсортируем таблицу
financial_prolongations.sort_values(by='payment_date', ascending=False, inplace=True)

# создадим функцию для замены нулевых значений
def fill_zero_payments(row, df):
    if row['payment'] == 0:
        # находим индекс нулевого значения
        id_zero = row['id']
        # находим месяц завершения проекта
        months = row['month']
        # находим предыдущую дату
        prev_date = row['payment_date'] - pd.DateOffset(months=1)
        # находим строки с нужным id и месяцем
        new_df = df[(df['id']==id_zero) & (df['month']==months)]
        # ищем отгрузку за предыдущий месяц 
        prev_payment = new_df[new_df['payment_date'] == prev_date]['payment']
        # если таблица будет не пустой, то заменим ноль на продажи предыдущего месяца
        if not prev_payment.empty:
            return prev_payment.iloc[0]
    # иначе оставляем нулём
    return row['payment']

# меняем нули
financial_prolongations['payment'] = financial_prolongations.apply(lambda x: fill_zero_payments(x, financial_prolongations), axis=1)

# 4. Нахождение коэффициентов

То есть, если нам нужно понять, насколько хорошо менеджер пролонгировал в мае, необходимо посчитать:
* Сумму отгрузки проектов, завершившихся в апреле (за апрель) и сумму отгрузки тех проектов завершившихся в апреле, у которых есть отгрузка в мае (за май). Коэффициент – отношение второй суммы к первой. 
* Сумму проектов, завершившихся в марте, у которых нет отгрузки в апреле (за март) и сумму отгрузки тех проектов, завершившихся в марте, у которых нет отгрузки в апреле но есть в мае (за май). Коэффициент – отношение второй суммы к первой. 

In [32]:
# в эти пустые словари будем добавлять коэффициенты, и не только
dict_k_1 = {}
dict_k_2 = {}

# пройдемся по менеджерам
for manager in financial_prolongations['AM'].unique():
    # сделаем срез по конкретному менеджеру manager
    manager_df = financial_prolongations[financial_prolongations['AM']==manager]
    # добавим в словарь в качестве ключа менеджера, а значением пустой список
    # в который будем добавлять всю инфомрацию по менеджерам, включая коэффициенты
    dict_k_1[manager] = list()
    dict_k_2[manager] = list()
    
    # пройдемся теперь по уникальным отсортированным датам
    for current_month in pd.Series(financial_prolongations.month.unique()).sort_values():
        # находим предыдущий месяц
        last_month = current_month - pd.DateOffset(months=1)
        # находим дважды предыдущий месяц для второго коэффицента
        last_last_month = current_month - pd.DateOffset(months=2)
        
        # если предыдущий месяц присутствует в срезе данных менеджера, то
        if manager_df['month'].isin([last_month]).any():
            # делаем срез отгрузок за предыдущий месяц 
            df_last = manager_df[(manager_df['month']==last_month) & (manager_df['payment_date']==last_month)]
            # сохраняем id этого среза
            id_last = df_last['id']
            # делаем еще один срез, но с отгрузками текущего месяца 
            df_current = manager_df[(manager_df['month']==last_month) & (manager_df['payment_date']==current_month) & (manager_df['id'].isin(id_last))]

            # осталось проверить, если оба среза не пустые, то 
            if not (df_last.empty or df_current.empty):
                # находим сумму отгрузок за предыдущий месяц
                revenue_1_month_ago = df_last['payment'].sum()
                # находим сумму отгрузок за текущий месяц
                revenue_current_month = df_current['payment'].sum()
                # и рассчитваем первый коэффицент пролонгации
                k_1 = revenue_current_month/revenue_1_month_ago

                # добавляем в словарь шесть значений: 
                # last_month - дата завршения проекта
                # revenue_1_month_ago - отгрузки в месяц завершения проекта
                # manager - менеджер
                # current_month - дата расчета пролонгации
                # revenue_current_month - отгрузки в месяц расчета коэффициента
                # k_1 - коэффициент пролонгации
                dict_k_1[manager].append([last_month, revenue_1_month_ago, manager, current_month, revenue_current_month, k_1])
        
        # по аналогии рассчитваем второй коэффицент
        if manager_df['month'].isin([last_last_month]).any():
            df_last_last = manager_df[(manager_df['month']==last_last_month) & (manager_df['payment_date']==last_last_month)]
            id_last_last = df_last_last['id']
            df_current = manager_df[(manager_df['month']==last_last_month) & (manager_df['payment_date']==current_month) & (manager_df['id'].isin(id_last_last))]
            if not (df_last_last.empty or df_current.empty):
                revenue_2_month_ago = df_last_last['payment'].sum()
                revenue_current_month = df_current['payment'].sum()
                k_2 = revenue_current_month/revenue_2_month_ago
                dict_k_2[manager].append([last_month, revenue_2_month_ago, manager, current_month, revenue_current_month, k_2])

In [33]:
# посмотрим на полученный словарь
display(dict_k_1)
display(dict_k_2)

{'Иванова Мария Сергеевна': [[Timestamp('2022-12-01 00:00:00'),
   np.float64(2296595.71),
   'Иванова Мария Сергеевна',
   Timestamp('2023-01-01 00:00:00'),
   np.float64(620231.92),
   np.float64(0.2700657835853922)],
  [Timestamp('2023-02-01 00:00:00'),
   np.float64(229633.75),
   'Иванова Мария Сергеевна',
   Timestamp('2023-03-01 00:00:00'),
   np.float64(102433.75),
   np.float64(0.4460744555188425)],
  [Timestamp('2023-03-01 00:00:00'),
   np.float64(821973.75),
   'Иванова Мария Сергеевна',
   Timestamp('2023-04-01 00:00:00'),
   np.float64(228248.0),
   np.float64(0.2776828335454751)],
  [Timestamp('2023-04-01 00:00:00'),
   np.float64(138158.0),
   'Иванова Мария Сергеевна',
   Timestamp('2023-05-01 00:00:00'),
   np.float64(138158.0),
   np.float64(1.0)],
  [Timestamp('2023-06-01 00:00:00'),
   np.float64(424292.25),
   'Иванова Мария Сергеевна',
   Timestamp('2023-07-01 00:00:00'),
   np.float64(200525.0),
   np.float64(0.4726105650056064)],
  [Timestamp('2023-07-01 00:00:

{'Иванова Мария Сергеевна': [[Timestamp('2023-01-01 00:00:00'),
   np.float64(2296595.71),
   'Иванова Мария Сергеевна',
   Timestamp('2023-02-01 00:00:00'),
   np.float64(473640.25),
   np.float64(0.20623579846363121)],
  [Timestamp('2023-03-01 00:00:00'),
   np.float64(229633.75),
   'Иванова Мария Сергеевна',
   Timestamp('2023-04-01 00:00:00'),
   np.float64(138158.0),
   np.float64(0.6016450108052497)],
  [Timestamp('2023-04-01 00:00:00'),
   np.float64(821973.75),
   'Иванова Мария Сергеевна',
   Timestamp('2023-05-01 00:00:00'),
   np.float64(255248.0),
   np.float64(0.31053059784451753)],
  [Timestamp('2023-05-01 00:00:00'),
   np.float64(138158.0),
   'Иванова Мария Сергеевна',
   Timestamp('2023-06-01 00:00:00'),
   np.float64(102433.75),
   np.float64(0.741424673200249)],
  [Timestamp('2023-07-01 00:00:00'),
   np.float64(424292.25),
   'Иванова Мария Сергеевна',
   Timestamp('2023-08-01 00:00:00'),
   np.float64(214415.0),
   np.float64(0.5053474344629203)],
  [Timestamp('2

Сформируем новую таблицу из первого полученного словаря.

In [34]:
# создаем пустой список для хранения данных
data_list_1 = []

# перебираем все записи в словаре
for manager, records in dict_k_1.items():
    for record in records:
        # извлекаем значения в нужном порядке
        last_month = record[0]
        revenue_1_month_ago = record[1]
        manager = record[2]
        current_month = record[3]
        revenue_current_month = record[4]
        k_1 = record[5]
        
        # добавляем запись в общий список
        data_list_1.append({
            'manager': manager,
            'last_month': last_month,
            'revenue_1_month_ago': revenue_1_month_ago,
            'current_month': current_month,
            'revenue_current_month': revenue_current_month,
            'k_1': k_1,
            'year': current_month.year
        })

# создаем датафрейм
df_1 = pd.DataFrame(data_list_1)

# сортируем по менеджеру и дате платежа
df_1.sort_values(by=['manager', 'last_month'], inplace=True)

# отображаем результат
df_1.head()

,manager,last_month,revenue_1_month_ago,current_month,revenue_current_month,k_1,year
36,Васильев Артем Александрович,2022-11-01,582479.00,2022-12-01,254545.00,0.437003,2022
37,Васильев Артем Александрович,2022-12-01,1663970.18,2023-01-01,1012761.60,0.608642,2023
38,Васильев Артем Александрович,2023-01-01,827225.00,2023-02-01,874015.00,1.056563,2023
39,Васильев Артем Александрович,2023-02-01,991467.42,2023-03-01,621897.55,0.627250,2023
40,Васильев Артем Александрович,2023-03-01,796055.45,2023-04-01,152640.00,0.191745,2023


Сформируем новую таблицу из второго полученного словаря.

In [35]:
data_list_2 = []

for manager, records in dict_k_2.items():
    for record in records:
        last_last_month = record[0]
        revenue_2_month_ago = record[1]
        manager = record[2]
        current_month = record[3]
        revenue_current_month = record[4]
        k_2 = record[5]
        
        data_list_2.append({
            'manager': manager,
            'last_last_month': last_last_month,
            'revenue_2_month_ago': revenue_2_month_ago,
            'current_month': current_month,
            'revenue_current_month': revenue_current_month,
            'k_2': k_2,
            'year': current_month.year
        })

df_2 = pd.DataFrame(data_list_2)

df_2.sort_values(by=['manager', 'last_last_month'], inplace=True)

df_2.head()

,manager,last_last_month,revenue_2_month_ago,current_month,revenue_current_month,k_2,year
37,Васильев Артем Александрович,2022-12-01,582479.00,2023-01-01,351740.00,0.603867,2023
38,Васильев Артем Александрович,2023-01-01,1663970.18,2023-02-01,1144844.69,0.688020,2023
39,Васильев Артем Александрович,2023-02-01,827225.00,2023-03-01,982114.08,1.187239,2023
40,Васильев Артем Александрович,2023-03-01,991467.42,2023-04-01,609954.00,0.615203,2023
41,Васильев Артем Александрович,2023-04-01,796055.45,2023-05-01,113617.30,0.142725,2023


# 5. Формирование файлов

Выгрузим отчет по первому коэффиценту.

In [36]:
# коэффициент пролонгации для каждого менеджера по месяцам
manager_month_1 = pd.pivot_table(df_1, values='k_1', index='manager', columns='current_month', aggfunc='mean')
# коэффициент пролонгации для каждого менеджера по годам
manager_year_1 = pd.pivot_table(df_1, values='k_1', index='manager', columns='year', aggfunc='mean')
# коэффициент пролонгации для отдела по месяца
department_month_1 = pd.pivot_table(df_1, values='k_1', columns='current_month', aggfunc='mean').rename(index={'k_1': 'Отдел'})
# коэффициент пролонгации для отдела по годам
department_year_1 = pd.pivot_table(df_1, values='k_1', columns='year', aggfunc='mean').rename(index={'k_1': 'Отдел'})

# сохраним файлы в формате csv
manager_month_1.to_csv('data/report/manager_month_1.csv')
manager_year_1.to_csv('data/report/manager_year_1.csv')
department_month_1.to_csv('data/report/department_month_1.csv')
department_year_1.to_csv('data/report/department_year_1.csv')

Выгрузим отчет по второму коэффиценту.

In [37]:
# коэффициент пролонгации для каждого менеджера по месяцам
manager_month_2 = pd.pivot_table(df_2, values='k_2', index='manager', columns='current_month', aggfunc='mean')
# коэффициент пролонгации для каждого менеджера по годам
manager_year_2 = pd.pivot_table(df_2, values='k_2', index='manager', columns='year', aggfunc='mean')
# коэффициент пролонгации для отдела по месяца
department_month_2 = pd.pivot_table(df_2, values='k_2', columns='current_month', aggfunc='mean').rename(index={'k_2': 'Отдел'})
# коэффициент пролонгации для отдела по годам
department_year_2 = pd.pivot_table(df_2, values='k_2', columns='year', aggfunc='mean').rename(index={'k_2': 'Отдел'})

# сохраним файлы в формате csv
manager_month_2.to_csv('data/report/manager_month_2.csv')
manager_year_2.to_csv('data/report/manager_year_2.csv')
department_month_2.to_csv('data/report/department_month_2.csv')
department_year_2.to_csv('data/report/department_year_2.csv')

Сохраним общую таблицу для дашборда.

In [38]:
df_1.to_csv('data/report/k_1.csv')
df_2.to_csv('data/report/k_2.csv')